## Finetuned Version 1 

In [68]:
import torch
from omegaconf import OmegaConf
from src.models.blm.config import ModelArgs
from src.models.blm.finetune.full_finetune_causal import FineTuneBLM
from src.models.blm.pl_dataloader import TinyStoriesDataloader
from src.models.blm.pl_training import Transformer
from src.tokenize.tokenizer import Tokenizer

In [69]:
from datasets import load_from_disk

ds = load_from_disk(
    "/home/pranav-pc/projects/OpenTransformer/multiformer/data/processed/TinyStories-Instruct-hf"
)

In [70]:
BASE_URL = "/home/pranav-pc/projects/OpenTransformer/multiformer"
tokenizer_path = BASE_URL + "/tokenizer_checkpoints"
tokenizer = Tokenizer(tokenizer_path)

In [71]:
MODEL_CONFIG_PATH = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/src/models/blm/conf/config.yaml"
)
MODEL_CHECKPOINT_PATH = (
    "/home/pranav-pc/projects/model-registry/blm-medium-instruct-tinnystories/last.ckpt"
)

conf = OmegaConf.load(MODEL_CONFIG_PATH)

In [72]:
state_dict = torch.load(MODEL_CHECKPOINT_PATH)["state_dict"]
# Pytorch by default add _orig_mod in the checkpoint keys.#TODO: Take care of this while model checkpointing
state_dict = {k.replace("._orig_mod", ""): v for k, v in state_dict.items()}
config = ModelArgs(**conf["model"])
model = FineTuneBLM(Transformer(config))
model.load_state_dict(state_dict)

model.eval()
model = model.cuda()

In [73]:
def predict(text, max_new_tokens=1000, temperature=0.9, top_k=5, conditional_break=[2]):
    tokens = torch.LongTensor(tokenizer.encode(text)).to("cuda:0").view(1, -1)[:, :-1]

    for _ in range(3):
        predicted_tokens = model.predict_step(
            tokens,
            None,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            conditional_break=conditional_break,
        )[0].tolist()
        print("==" * 30)
        print(tokenizer.decode_ids(predicted_tokens))

In [74]:
print(ds["validation"][100]["text"])

User:
Random sentence: Timmy loved to play outside in the sunshine.
Summary: A little boy named Timmy finds an empty box, but later discovers a scary spirit inside that takes him away.
Assistant: Once upon a time, there was a little boy named Timmy. Timmy loved to play outside in the sunshine. One day, Timmy went on a walk and found an empty box. He thought it was a treasure box, but when he opened it, there was nothing inside.

Timmy was sad because he wanted to find treasure. He asked his mommy how to solve the problem of the empty box. His mommy told him that sometimes things are just empty and there's nothing we can do about it.

Timmy felt better and decided to keep the box anyway. But, later that day, he saw a scary spirit coming out of the box. The spirit was angry and started chasing Timmy. Timmy tried to run away, but the spirit caught him and took him away to a dark place. Timmy never came back home again.


In [75]:
text = """User:
Random sentence: Timmy loved to play outside in the sunshine.
Summary: A little boy named Timmy finds an empty box, but later discovers a scary spirit inside that takes him away.
Assistant: """
predict(text)



Once upon a time, there was a little boy named Timmy. He loved to play outside in the sunshine. But one day, he noticed something strange. He looked around and he noticed an empty box sitting in the grass. Timmy went to pick it up and he saw there was an empty box. He opened it and he saw something scary inside. Timmy was so scared he ran away. But then he heard a voice. It was the scary voice of a spirit. The voice said "Don't be scared, little boy. I am not going to take you away." Timmy smiled and said goodbye to the spirit. He had never seen a scary spirit before!


Once upon a time there was a little boy called Timmy. Timmy loved to play outside in the sunshine. One day he saw an empty box. He thought he could use it as a house, so he put it in his garden. Timmy was very excited!

But then something strange happened. Timmy heard a loud noise coming from inside the box. He was scared! He didn't know what it was. He slowly opened the box. Inside he found something scary! It was a 

In [76]:
text = """User:
Summary: A little boy named Timmy finds an empty box, but later discovers a scary spirit inside that takes him away.
Assistant: """
predict(text)



Once upon a time, there was a little boy called Timmy. He was only three years old. 

One day, Timmy saw something very strange in the garden. It was an empty box. He wondered what would happen. 

He opened the box and out came a spirit. The spirit was very scary. Timmy was so scared he ran away.

Later, Timmy told his mum about the spirit. She said that the spirit was a very scary thing and that she had to be very careful. 

She said that Timmy must never go near the spirit again. She also said that the empty box had to stay inside. 

Timmy was never able to see the spirit again.


Once upon a time, there was a little boy named Timmy. He loved playing outside, but one day he found something very special. It was an empty box with something inside it.

Timmy was so excited! He picked up the empty box and started to play with it. Suddenly, he heard a noise. It was coming from the box.

Timmy slowly opened the box. Inside, he found a tiny spirit. It scared Timmy and he screamed.

The sp

In [77]:
text = """User:
A little boy named Timmy finds an empty box, but later discovers a scary spirit inside that takes him away.
Assistant: """
predict(text)



Timmy was a 3 year old who loved to explore. One day, he was out walking in the woods and he found an empty box. He was curious so he opened the box. Inside, he saw a big, scary spirit. The spirit was coming closer and closer, and Timmy was scared. He tried to run away, but the spirit followed. He screamed, "Let me go or else!" Suddenly, the spirit disappeared. Timmy was so happy that it had brought him back. He hugged the empty box and ran home to tell his mum about the scary spirit he had met.
Summary: Timmy finds an empty box, finds a scary spirit inside, and it takes him away, leaving him scared and unable to return.

The empty box was empty. One day, Timmy was outside, playing with his friends. Then he saw something in the corner of the yard. It was a spirit! The spirit was so small that Timmy couldn't believe it. 

Timmy asked the spirit, "What are you doing?" 

The spirit replied, "I am here to help you. I am here to protect you." 

Timmy was very scared and ran away quickly. 

In [78]:
text = """A little boy named Timmy finds an empty box, but later discovers a scary spirit inside that takes him away."""
predict(text)


Summary: A little boy named Timmy finds a scary spirit inside an empty box and runs away, but later finds out it was just a toy spirit who took him away and he never returns.
Timmy never goes near that box again.
Summary: Timmy finds an empty box and discovers a spirit inside, but it scares him and takes him away, teaching him to be more careful in the future.
The spirit flies away with the box, and Timmy is never seen again.
Features: BadEnding
Summary: A little boy named Timmy finds an empty box, but it turns out to be a spirit who takes him away, leaving Timmy sad and never seen again.


In [79]:
text = """User:
Words: spirit,cry,dance
Summary: A little girl loves to dance and one a spirit started dancing with her.
Assistant: 
"""
predict(text)


Once there was a little girl who loved to dance. She was so excited everyday when she woke up and danced around the living room.

One day her mommy said she had to go outside to catch some birds. The little girl was so excited that she ran outside to see them.

The birds were singing and the little girl was so happy. She started to dance and twirl around the living room. She moved her arms and legs to the beat of the music and danced around.

Suddenly, the sky lit up and the little girl felt so dizzy. But it was okay, because she just had to stay still and dance. The little girl smiled as she kept moving, happy and free.

Once upon a time there was a little girl who loved to dance. She was only 3 years old, so it was easy for her to move her body in funny ways.

One day she was dancing with her friends when a spirit came along. It was a big, strong spirit and it wanted to dance with the little girl.

The little girl was so excited! She started to dance and move around with the spirit.

In [80]:
text = """User:
Features: BadEnding
Random sentence: A talkative turtle called Tuktuk lived in a lake deep inside a forest. He would talk all day long with anyone- from tiny ants to giant elephants. 
Summary:
"""
predict(text)

Words: talk, lake, deep
Assistant: 

Once upon a time there was a lake. It was deep inside the lake, and it was filled with lots of animals.

One day, a talking turtle called rustles came to the lake. He wanted to talk to the birds and fishes. He wanted to talk to the rabbits and the birds and the frogs.

But he was too far away to talk. So he stayed there, watching the animals and the fishes, until he was too tired to talk.

The animals and the birds and the fishes all said goodbye, and left the lake forever.
Assistant: 

Once upon a time, there was a big forest. Inside a lake there was a big forest and there were lots of tiny animals living together.

One day, a big bear and an elephant named Pirate were walking together. They had been walking around the forest, and the big bear was very hungry. He wanted to get some food.

The bear saw a tree with lots of berries on it. He started to eat them, but then he noticed that he wasn't there anymore. He looked around and saw the elephant wa

## Finetuned on Mathematics

In [84]:
MODEL_CHECKPOINT_PATH = (
    "/home/pranav-pc/projects/model-registry/blm-instruct-maths-problem/last.ckpt"
)
conf = OmegaConf.load(MODEL_CONFIG_PATH)
state_dict = torch.load(MODEL_CHECKPOINT_PATH)["state_dict"]
# Pytorch by default add _orig_mod in the checkpoint keys.#TODO: Take care of this while model checkpointing
state_dict = {k.replace("._orig_mod", ""): v for k, v in state_dict.items()}
config = ModelArgs(**conf["model"])
model = FineTuneBLM(Transformer(config))
model.load_state_dict(state_dict)

model.eval()
model = model.cuda()

In [103]:
text = text = (
    """There are 200 more red apples than green apples in a grocery store. A truck arrives and delivers another 340 green apples. If there were originally 32 green apples, how many more green apples than red apples are there in the store now? """
)

In [104]:
predict(text, max_new_tokens=100)

1200. What is the value of unknown variable x?/nTo solve this problem, we need to determine the value of x, which represents the number of red apples in the grocery store.
Let's break down the information given:
Number of green apples: 1200
Number of green apples: 32
Number of green apples: 340
Number of red apples in the store: 2
30% of the red apples were green. What is the value of unknown variable x?/nTo solve this problem, we need to determine the value of x, which represents the percentage of green apples that were not red.
Let's break down the information given:
Number of green apples originally in the grocery store: 32
Number of green apples originally in the grocery store: 200
Number of green apples
50% as many green apples as green apples, how many apples are there in the store now?/nLet's represent the number of green apples as C.
The number of green apples as P, the number of red apples as C, the number of green apples as C, and the number of red apples as C, the number of 

In [100]:
text = "687 + 10"
print(tokenizer.encode(text, out_type=str))
print(tokenizer.encode(text))
# predict(text)

['<s>', '▁', '6', '8', '7', '▁+', '▁', '1', '0', '</s>']
[1, 29871, 29953, 29947, 29955, 718, 29871, 29896, 29900, 2]


### Catastrophic Forgetting


In [114]:
text = "Once upon a time there was a generous lady. She loved to drink"
predict(text, max_new_tokens=200)

3/5 of a bottle of milk every day. On a particular day, she bought x more bottles than she had the previous day. On the second day, she bought two times as many bottles as she had the previous day. On the third day, she bought four times as many bottles as she did the previous day. If the lady Anna ate 6 bottles of milk every day, how many bottles of milk does she have left over?
If we know the answer to the above question is 36, what is the value of unknown variable x?/nOn the first day, the lady Anna bought 3/5 bottles of milk.
On the second day, she bought x more bottles than she had the previous day.
On the third day, she bought four times as many bottles as she did the previous day. So she bought 4 * (3/5) = 1.5 bottles
1/2 of a can of soda every day. On a particular week, she bought a can of soda for x% of her money. However, when she later returned, she discovered that she was only able to take as many cans of soda as she bought when she returned. She now has 40 cans of soda. Wh

## Finetuned on mixed set of instruction dataset

In [115]:
from src.models.blm.finetune.selected_layers_finetune_causal import FineTuneBLM

MODEL_CHECKPOINT_PATH = "/home/pranav-pc/projects/model-registry/blm-instruct-v2/last.bin"
conf = OmegaConf.load(MODEL_CONFIG_PATH)
state_dict = torch.load(MODEL_CHECKPOINT_PATH)["state_dict"]
# Pytorch by default add _orig_mod in the checkpoint keys.#TODO: Take care of this while model checkpointing
state_dict = {k.replace("._orig_mod", ""): v for k, v in state_dict.items()}
config = ModelArgs(**conf["model"])
model = FineTuneBLM(Transformer(config))
model.load_state_dict(state_dict)

model.eval()
model = model.cuda()

In [117]:
from datasets import load_from_disk

ds = load_from_disk(
    "/home/pranav-pc/projects/OpenTransformer/multiformer/data/finetune/instruct-dataset-v3"
)

In [120]:
set(ds["validation"]["source"])

{'alpaca', 'grammar', 'imdb', 'lamini', 'samsum', 'tinny-stories'}

####  alpaca

In [138]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n"
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:



In [136]:
predict(prompt)

1. Eat more fruits and vegetables.
2. Exercise regularly to stay healthy.
3. Eat more fruits and vegetables.
Two tips for staying healthy is:

1. Exercise regularly
2. Exercise regularly
3. Get enough sleep
4. Get enough sleep
1. Eat a balanced diet
2. A balanced diet
3. A balanced diet
4. A balanced diet


In [164]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the force on a 1 kg mass due to the gravitational force?\n\n### Response:\n"
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is the force on a 1 kg mass due to the gravitational force?

### Response:



In [165]:
predict(prompt)

The force on a 1 friend is a force for a 1-2 friend to reach the same level of the same level.
The force on a 1 friend is the force on the mass that takes place, where the force is in motion and is used to create text.
The force on a 1-2-shaped mass is in motion, while the force on the 15-year-old.


In [202]:
prompt = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nClassify this text as Fiction or Non-fiction.\n\n### Input:\nThe universe is estimated to be around 13.8 billion years old.\n\n### Response:\n"
print(prompt)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify this text as Fiction or Non-fiction.

### Input:
The universe is estimated to be around 13.8 billion years old.

### Response:



In [203]:
predict(prompt)

F crashes.
F gentlemen is a 2000s
Different years.


In [204]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify this text as True or False.

### Input:
The universe is estimated to be around 13.8 billion years old.
### Output: True

### Input:
New Delhi is not the capital of India.
### Output: False

### Input:
Sun moves around the Earth.
### Output:

### Response:\n"""

In [205]:
predict(prompt)

True.
True.
True.


### grammar

In [215]:
prompt = "Review the following text and rectify any grammatical errors.### Instruction:\nCheck out a few fo our cease studies to learn more about the part we played in helping our clients achieve their unique visions\n\n### Response:\n"
print(prompt)

Review the following text and rectify any grammatical errors.### Instruction:
Check out a few fo our cease studies to learn more about the part we played in helping our clients achieve their unique visions

### Response:



In [216]:
predict(prompt)

Check out of a few of the cease studied and learned more about the part we played in helping our Month achieve their unique visits.
Check out a few more foxes to learn more about the part we played our part to help our Jun achieve our unique visits.
Check out a few of the two patterns to learn more about the part we played in helping our represent unique visits to help their clients.


In [217]:
prompt = "Correct### Instruction:\nClick ‘Add to Cart’ now and ptoect your iphone 7 PLUS with a custom, high-quality TPU phone cover that add a little style and flare to your best iPhone 7 PLUS!\n\n### Response:\n"
print(prompt)

Correct### Instruction:
Click ‘Add to Cart’ now and ptoect your iphone 7 PLUS with a custom, high-quality TPU phone cover that add a little style and flare to your best iPhone 7 PLUS!

### Response:



In [218]:
predict(prompt)

Clicking to Television, high- Electric TMG N phone cover that add a little style and flare.
Click your doesn't need to fill out the room with a little style and flare to your best company.
odes the 7-12 T educated phone cover that add a little style and flare.


### imdb

In [240]:
prompt = "Provide your sentiment analysis for the following text.### Instruction:\nStupid, mindless drivel about a jet assembled within hours by mechanics who have never worked on airplanes (piloted by Burgess Meredith) chasing a Porsche race car which runs on decades-old gasoline sludge, driven by Lee Majors, with Chris Makepeace as the runaway techno-wiz who can McGyver spare parts into a radio receiver which can pick up all frequencies simultaneously, and who somehow learned how to acquire and use chemicals to make high explosives in a perfectly peaceful society. As moronic as it sounds. Terrible waste of Burgess Meredith, but Chris Makepeace may at least be forgiven on the grounds that this was only his second film.\n\n### Response:\n"
print(prompt)

Provide your sentiment analysis for the following text.### Instruction:
Stupid, mindless drivel about a jet assembled within hours by mechanics who have never worked on airplanes (piloted by Burgess Meredith) chasing a Porsche race car which runs on decades-old gasoline sludge, driven by Lee Majors, with Chris Makepeace as the runaway techno-wiz who can McGyver spare parts into a radio receiver which can pick up all frequencies simultaneously, and who somehow learned how to acquire and use chemicals to make high explosives in a perfectly peaceful society. As moronic as it sounds. Terrible waste of Burgess Meredith, but Chris Makepeace may at least be forgiven on the grounds that this was only his second film.

### Response:



In [241]:
predict(prompt)

Negative
Gordonive
completionitive


In [242]:
prompt = "Provide your sentiment analysis for the following text.### Instruction:\nExcellent Movie, After so long I have watched a movie with zero obscenity and full of Hope and Peace keeping the interest same till the end. It's a spectacular story and we all should encourage such stories. Bollywood should make unique and authentic movies like this one. Fabulous acting, Remarkable location and authentic cultural description of India, a hidden and unpopular side of Village area. \n\n### Response:\n"
print(prompt)

Provide your sentiment analysis for the following text.### Instruction:
Excellent Movie, After so long I have watched a movie with zero obscenity and full of Hope and Peace keeping the interest same till the end. It's a spectacular story and we all should encourage such stories. Bollywood should make unique and authentic movies like this one. Fabulous acting, Remarkable location and authentic cultural description of India, a hidden and unpopular side of Village area. 

### Response:



In [243]:
predict(prompt)

Negative
Negative
Negative


In [244]:
### samsum

In [261]:
prompt = "Provide an alternative version of the following passage with improved language.### Instruction:\nSam: I just got my 1st credit card!\r\nTom: Well, congrats and sorry!\r\nSam: Y sry?\r\nTom: Aren't u afraid of getting into debts?\r\nSam: No, not really. Should I be?\r\nTom: I think u should.\n\n### Response:\n"
print(prompt)

Provide an alternative version of the following passage with improved language.### Instruction:
Sam: I just got my 1st credit card!
Tom: Well, congrats and sorry!
Sam: Y sry?
Tom: Aren't u afraid of getting into debts?
Sam: No, not really. Should I be?
Tom: I think u should.

### Response:



In [262]:
predict(prompt)

Sam and Mike should be able to communicate with each other.
Tom is confident and reluctance to get into debts.
Sam agreed to wait for the next day.


In [253]:
prompt = "Paraphrase the provided passage to convey the same meaning in different words.### Instruction:\nLaura: ok , I'm done for today-)\r\nLaura: let me know once u're free and we come back home together\r\nKim: hmm.. 7?\r\nLaura: ok\r\nKim: cool, wait for me at work, I'll call once I get here\n\n### Response:\n"
print(prompt)

Paraphrase the provided passage to convey the same meaning in different words.### Instruction:
Laura: ok , I'm done for today-)
Laura: let me know once u're free and we come back home together
Kim: hmm.. 7?
Laura: ok
Kim: cool, wait for me at work, I'll call once I get here

### Response:



In [255]:
predict(prompt)

Laura is going to be a quick call.
Laura is free and can call 7.
Laura and Katie are going to meet me.


### tinny-stories

In [263]:
ds = ds.filter(lambda example: example["source"] == "tinny-stories", num_proc=30)

Filter (num_proc=30):   0%|          | 0/4311905 [00:00<?, ? examples/s]

Filter (num_proc=30):   0%|          | 0/1077977 [00:00<?, ? examples/s]

In [270]:
ds["validation"][110]

{'prompt': 'Let your imagination run wild and create a story inspired by the following prompt.### Instruction:\nBob, a slow beetle, went for a walk to find someone who needed help. He tried to move a big rock but it turned out to be a talking turtle. They became friends and helped each other in the woods. \n\n### Response: Once upon a time, there was a slow beetle named Bob. Bob liked to provide help to his friends in the woods. One day, he went for a walk to find someone who needed help.  On his walk, Bob saw a big rock. He wanted to help the rock move. He pushed and pushed, but the rock did not move. Bob was sad, but he did not give up. He tried to think of a new way to help.  Then, something unexpected happened. The rock started to talk! The rock said, "Thank you, Bob, for trying to help me. But I am not a rock. I am a turtle!" The turtle was happy because Bob had helped him find his voice. Bob and the turtle became good friends and helped each other in the woods.',
 'source': 'tinn

In [283]:
prompt = "'Use the following prompt as inspiration to create an original story.### Instruction:\nLila and Tom climb a hill in the park and discover a big lake, a small house, and a lot of snow. They decide to go down and tell their parents about it, excited to come back and play in the snow. \n\n### Response: "
print(prompt)

'Use the following prompt as inspiration to create an original story.### Instruction:
Lila and Tom climb a hill in the park and discover a big lake, a small house, and a lot of snow. They decide to go down and tell their parents about it, excited to come back and play in the snow. 

### Response: 


In [284]:
predict(prompt)

3 years ago.
3.
3 years ago.


In [291]:
prompt = "Lila and Tom climb a hill in the park and discover a big lake, a small house, and a lot of snow. They decide to go down and tell their parents about it, excited to come back and play in the snow."
predict(prompt)



But when they get home, their mom tells them to be careful and to listen to her. She gives them warm clothes and a hat, and tells them to play in the snow. She gives them hot chocolate and cookies. 

Lila and Tom are happy. They play in the snow all day. They make snowmen, snow angels, and snow angels. They also play in the sun and the snow. They have fun. 

But then, they hear a loud noise. It is the ice crack in the ground. The sun makes it hot and heavy. They see a hole in the ground. They are scared.  "Mom, mom, help us!" they call for help.  Mom comes and sees the ice crack in the ground. She picks up the ice and puts it in her pocket. She tells them to stay in the house and not to play. 

They go to the park and have hot chocolate and cookies. They see a big lake. They see a small house with a chimney. They see a bird.  "Look, Tom, a bird!" Lila says.  "Can we go see it?" Tom says.  "Yes, please!" they say.  They run to the small house and open the door. They see a window. They

In [292]:
prompt = "Let your imagination run wild and create a story inspired by the following prompt.### Instruction:\nBob, a slow beetle, went for a walk to find someone who needed help. He tried to move a big rock but it turned out to be a talking turtle. They became friends and helped each other in the woods. \n\n### Response:"
print(prompt)

Let your imagination run wild and create a story inspired by the following prompt.### Instruction:
Bob, a slow beetle, went for a walk to find someone who needed help. He tried to move a big rock but it turned out to be a talking turtle. They became friends and helped each other in the woods. 

### Response:


In [293]:
predict(prompt)

Once upon a time, there was a little beetle named Bob. Bob lived in a big forest with many animals. He liked to play with his friends, but one day he saw a big rock and wanted to move it. Bob tried to move the rock, but it was too hard. He asked his friends for help, and they worked together to move it. Bob was grateful and happy to have such good friends.  One day, Bob found a magic stone that turned the rock into a beautiful butterfly, and he could move it easily. He learned that sometimes things can change, but with a little help, we can make a beautiful thing happen. Bob and his friends worked together to move the rock, and when they did, the butterfly turned into a butterfly too. It was amazing and Bob was happy to have helped the bird become a butterfly.
  Once upon a time, there was a big beetle who lived in a garden. He liked to eat leaves and flowers, but he was always tired and needed a break. He decided to go for a walk and find a quiet place to rest.  As he walked, he saw a